In [1]:
# tk: making it easier to reference throughout the code.
import tkinter as tk
#  The * (asterisk) symbol means importing everything, although it's generally considered better practice to import only what you need to avoid namespace pollution.
from tkinter import *
# messagebox: This module provides methods to create pop-up message boxes for various purposes, such as showing information, warnings, errors, or asking for user input.
from tkinter import messagebox

In [26]:
window = tk.Tk()
window.title("Handwritten digit recognition")

l1 = tk.Label(window, text="Digit", font=('Algerian', 20))
l1.place(x=5, y=0)

t1 = tk.Entry(window, width=20, border=5)
t1.place(x=150, y = 0)

def screen_capture():
    import pyscreenshot as ImageGrab
    import time
    import os

    os.startfile("C:/Users/admin/AppData/Local/Microsoft/WindowsApps/Microsoft.Paint_8wekyb3d8bbwe/mspaint.exe")
    s1 = t1.get()
    os.chdir("D:/FPT/AI/Major4/Handwritten Digit Recognition/captured_images")
    os.mkdir(s1)
    os.chdir("D:/FPT/AI/Major4/Handwritten Digit Recognition/")

    images_folder = "captured_images/"+str(s1)+"/"
    time.sleep(15)
    for i in range(100,102):
        time.sleep(8)
        # PIL.ImageGrab.grab() method takes a snapshot of the screen. 
        # image from the screen starting at the coordinates (60, 170) on the screen and ending at the coordinates (400, 550)
        im = ImageGrab.grab(bbox=(60,170,400,550)) #x1, y1, x2, y2
        print("saved.......", i)
        # print(i, end=" ")
        # if(i%18==0):
        #     print()
        im.save(images_folder+str(i)+'.png')
        print("clear screen now and redraw now.......")
    messagebox.showinfo("Result","Capturing screen is completed!!")

b1 = tk.Button(window, text="1. Open pain and capture the screen", font=('Algerian', 15), bg="orange", fg="black", command=screen_capture)
b1.place(x=5, y=50)

def generate_dataset():
    import cv2
    import csv
    import glob

    header = ["label"]
    for i in range(0, 784):
        header.append("pixel" + str(i))
    with open('GUI_dataset.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(header)

    for label in range(10):
        dirList = glob.glob("captured_images/"+str(label)+"/*.png")

        for img_path in dirList:
            # cv2.imread() method loads an image from the specified file.
            im = cv2.imread(img_path)
            # cv2.cvtColor() method is used to convert an image from one color space to another
            im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            # applying Gaussian blur to an image
            im_gray = cv2.GaussianBlur(im_gray,(15,15), 0)
            roi = cv2.resize(im_gray, (28,28), interpolation = cv2.INTER_AREA)

            data = []
            data.append(label)
            rows, cols = roi.shape

            ## Add pixel one by one into data array
            for i in range(rows):
                for j in range(cols):
                    k = roi[i,j]
                    if k > 100:
                        k = 1
                    else:
                        k = 0
                    data.append(k)
            with open('GUI_dataset.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(data)
    messagebox.showinfo("Result", "Generating dataset is complete!!")
    
b2 = tk.Button(window, text="2. Generate dataset", font=('Algerian', 15), bg="pink", fg="blue", command=generate_dataset)
b2.place(x=5, y=100)

def train_save_accuracy():
    import pandas as pd
    from sklearn.utils import  shuffle
    data = pd.read_csv('GUI_dataset.csv')
    data = shuffle(data)
    X = data.drop(["label"], axis=1)
    Y = data["label"]
    from sklearn.model_selection import  train_test_split
    train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size = 0.2)
    # import joblib: Đây là một thư viện trong Python dùng để lưu trữ và tải các đối tượng Python, như các mô hình máy học đã được đào tạo.
    import joblib
    # SVC là một trong những thuật toán Support Vector Machine dùng để phân loại dữ liệu.
    from sklearn.svm import SVC
    #  sử dụng các hàm kernel để ánh xạ dữ liệu vào không gian cao chiều hơn để có thể tìm ra ranh giới phân chia tốt hơn trong không gian đó.
    classifier = SVC(kernel="linear", random_state = 6)
    classifier.fit(train_x, train_y)
    # thư viện Joblib trong Python để lưu trữ mô hình phân loại (classifier) 
    joblib.dump(classifier, "GUI_model/digit_recognizer")
    from sklearn import metrics
    prediction = classifier.predict(test_x)
    acc = metrics.accuracy_score(prediction, test_y)
    messagebox.showinfo("Result", f"Your accuracy is {acc}")

b3 = tk.Button(window, text="3. Train the model, save it and calculate accurace", font=('Algerian', 15), bg="green", fg="white", command= train_save_accuracy)
b3.place(x=5, y=150)

def prediction():
    import joblib
    import cv2
    import numpy as np
    import time
    import pyscreenshot as ImageGrab
    import os
    os.startfile("C:/Users/admin/AppData/Local/Microsoft/WindowsApps/Microsoft.Paint_8wekyb3d8bbwe/mspaint.exe")
    
    model = joblib.load("GUI_model/digit_recognizer")
    images_folder = "GUI_img/"

    time.sleep(20)
    while True:
        img = ImageGrab.grab(bbox=(60,170,400,550))

        img.save(images_folder + "GUI_img.png")
        im = cv2.imread(images_folder+"GUI_img.png")
        im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        # applying Gaussian blur to an image
        im_gray = cv2.GaussianBlur(im_gray,(15,15), 0)

        # Threshold the image
        # cv2.threshold là một hàm trong thư viện OpenCV trong Python được sử dụng để áp dụng phép ngưỡng hóa 
        # trên một ảnh xám (grayscale image). Hàm này nhận đầu vào là ảnh xám (im_gray) và áp dụng một ngưỡng 
        # để chuyển đổi ảnh thành ảnh nhị phân (cv2.THRESH_BINARY).
        # 100: Đây là giá trị ngưỡng. Tất cả các giá trị điểm ảnh lớn hơn 100 sẽ được chuyển đổi thành 255 (trắng).
        # 255: Giá trị tối đa được sử dụng sau khi áp dụng ngưỡng. Trong trường hợp này, nếu giá trị điểm ảnh lớn hơn ngưỡng, chúng sẽ được gán giá trị này (trắng).
        # cv2.THRESH_BINARY: Đây là loại ngưỡng hóa, trong trường hợp này, nó sẽ chuyển đổi ảnh thành ảnh nhị phân, tức là chỉ có hai giá trị điểm ảnh là 0 hoặc 255 (đen hoặc trắng).
        # ret: Là giá trị ngưỡng được sử dụng trong quá trình ngưỡng hóa. Trong trường hợp này, giá trị 100.
        # im_th: Là ảnh sau khi đã được áp dụng ngưỡng, tức là ảnh nhị phân, chỉ có hai màu đen và trắng, dựa trên ngưỡng được chỉ định.

        ret, im_th = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
        roi = cv2.resize(im_th, (28,28), interpolation = cv2.INTER_AREA)
        rows, cols = roi.shape

        X = []

        ## Add pixel one bt one into data arry
        for i in range(rows):
            for j in range(cols):
                k = roi[i,j]
                if(k>100): k = 1
                else: k = 0
                X.append(k)
        
        predictions = model.predict([X])
        print("prediction:", predictions[0])
        cv2.putText(im, "prediction is: " + str(predictions[0]), (20,20), 0, 0.8, (0,255,0), 2, cv2.LINE_AA)

        cv2.startWindowThread()
        cv2.namedWindow("Result:")
        cv2.imshow("Result", im)
        cv2.waitKey(10000)
        ## 27 in ascii value of esc
        ## 13 is the ascii value enter
        if cv2.waitKey(1) == 13:
            break

    cv2.destroyAllWindows()

b4 = tk.Button(window, text="4. Live prediction", font=('Algerian', 15), bg="white", fg="red", command= prediction)
b4.place(x=5, y=200)

window.geometry("600x300")
window.mainloop()